Raw data and code from https://drive.google.com/drive/folders/1Jsv34JjNo22NOCd26iBHMP80EPG0xmQE, linked from Casey Handmer's blog post [Solar and batteries for generic use cases](https://caseyhandmer.wordpress.com/2024/11/09/solar-and-batteries-for-generic-use-cases/)

In [ ]:
from datetime import datetime
from pathlib import Path
from random import uniform

import altair as alt
from numba import njit
import pandas as pd
from vega_datasets import data

alt.data_transformers.enable("vegafusion")

folder = Path('./texas')
folder.exists()

In [ ]:
actual = [fp for fp in folder.iterdir() if fp.stem.startswith('Actual')]
actual

In [ ]:
states = alt.topo_feature(data.us_10m.url, feature='states')

usa_states = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)
usa_states

In [ ]:
def stem_to_lat_lon(s: str) -> tuple[float, float]:
    return tuple(float(x) for x in s.split('_')[1:3])

df = pd.DataFrame([stem_to_lat_lon(fp.stem) for fp in actual], columns=['lat', 'lon'])

usa_states + alt.Chart(df).mark_circle().encode(
    longitude='lon:Q',
    latitude='lat:Q',
    size=alt.value(10),
).project(
    'albersUsa'
).properties(width=500, height=300)

In [ ]:
raw = pd.read_csv(folder / 'Actual_36.25_-102.95_2006_UPV_84MW_5_Min.csv')
raw['local_time'] = raw['LocalTime'].apply(lambda s: datetime.strptime(s, '%m/%d/%y %H:%M'))
raw['power'] = raw['Power(MW)']
raw = raw.drop(columns=['LocalTime', 'Power(MW)'])
raw.head()

In [ ]:
raw.describe()

In [ ]:
jan_1st = raw.loc[(raw['local_time'].dt.day == 1) & (raw['local_time'].dt.month == 1)]
alt.Chart(jan_1st).mark_line().encode(x='local_time', y='power', tooltip='local_time').properties(width=500)

In [ ]:
from datetime import date
day_avg = raw.groupby(raw['local_time'].dt.time).mean().drop(columns=['local_time']).reset_index()
day_avg['local_time'] = day_avg['local_time'].apply(lambda t: datetime.combine(date.today(), t))
alt.Chart(day_avg).mark_line().encode(x='local_time', y='power').properties(title="Average power generation over a day")

In [ ]:
from datetime import time

year_avg = raw.groupby(raw['local_time'].dt.date).mean().drop(columns=['local_time']).reset_index()
year_avg['local_time'] = year_avg['local_time'].apply(lambda t: datetime.combine(t, time(12, 0, 0)))
alt.Chart(year_avg).mark_line().encode(
    x='local_time', y='power'
).properties(height=500, width=800)

In [ ]:
sol = raw.copy()
sol['power'] /= 84  # 84 MW array, standardize on 1MW
sol['power'].mean(), len(sol)

In [ ]:
sol.head()

In [ ]:
alt.Chart(sol).mark_line().encode(
    x='local_time', y='power'
).properties(
    width=800, height=600
)

In [ ]:
days = raw['local_time'].dt.date.unique()[::10]
df = raw.loc[raw['local_time'].dt.date.isin(days)].assign(
    day=lambda x: x['local_time'].dt.date.apply(lambda y: datetime.combine(y, time(12, 0, 0))),
    time=lambda x: x['local_time'].dt.time.apply(lambda y: datetime.combine(date.today(), y)),
)
alt.Chart(df.sample(5000)).mark_line().encode(
    x="time:T",
    y="power:Q",
    color="day:N"
)

Time to implement the `uptime` function. We baseline a 1MW array, then set up numerical array with loads of different sizes and batteries of different sizes. If the battery is empty, load is off. If battery is full, no chargning can occur. We measure everything in 5 minute intervals (according to the data), and assume the battery starts full.
Battery state is measure in MWh stored, so in each interval we have to divide by 12.

Let's start with a naive variant (no vectorization)

In [ ]:
24*365

In [ ]:
# capacity in MWh, load in MW, sol in MW
@njit
def uptime(capacity: float, load: float, sol: list[float]) -> tuple[float, float, float, float]:
    battery: list[float] = [capacity] + [0.0 for _ in sol] # MWh
    utilization: list[float] = [0.0 for _ in sol]  # percentage
    t_interval = 24.0 * 365.0 / len(sol)  # hours
    for i in range(len(sol)):
        sol_interval: float = sol[i]
        remaining_load: float = (load - sol_interval)*t_interval
        if sol_interval > load:  # More sun than load
            utilization[i] = 1  # Can run full load
            excess_solar = sol_interval - load
            # if battery[i] < capacity:
                # battery[i+1] = battery[i] + t_interval*excess_solar
            battery[i+1] = min(battery[i] + t_interval*excess_solar, capacity)
            
        elif battery[i] > remaining_load: # Battery is full enough
            utilization[i] = 1
            battery[i+1] = battery[i] - remaining_load
        else: # Battery fully drained, cannot use all load
            utilization[i] = (sol_interval * t_interval + battery[i]) / (load * t_interval)  # equivalent: sol_interval / load + battery / (load * ts)
            battery[i+1] = 0

    batsum = 0
    for b in battery[:-1]:
        batsum += 1 if b > 0 else 0
        
    return (
        capacity,
        load,
        batsum / (len(battery) - 1),
        t_interval * sum(utilization) / (24 * 365),
    )

uptime(2.0, 1.0, sol['power'].tolist())

In [ ]:
@njit
def all_in_system_cost(
    solar_cost: float, battery_cost: float, load_cost: float, battery_size: float, array_size: float, sol: list[float]
) -> float:
    capacity, load, battery_util, total_util = uptime(
        capacity=battery_size / array_size,
        load=1 / array_size,
        sol=sol,
    )
    return (capacity * battery_cost + solar_cost + load_cost * load) / (load * total_util)

all_in_system_cost(200_000, 200_000, 5_000_000, 10, 5, sol['power'].tolist())

Casey's number here is `1.0523e7`, so I'm going to call that close enough.
There appears to be a bug in Casey's implementation. The check if the battery can be charged happens at the start of the interval. If the battery is almost full this will be negative, but then there might be too much solar excess, filling the battery beyond max.  However, if we leave the bug in the program, the number doesn't match quite as closely... So either this bug is not in Casey's program, or my program has another discrepancy.

In [ ]:
@njit
def cost_and_elasticity(
    solar_cost: float, battery_cost: float, load_cost: float, battery_size: float, array_size: float, sol: list[float]
) -> tuple[float, float, float, float, float]:
    size_to_cost = lambda b, a: all_in_system_cost(
        solar_cost, battery_cost, load_cost, b, a, sol
    )
    cost = size_to_cost(battery_size, array_size)
    cost_battery = size_to_cost(1.01*battery_size+0.01, array_size)
    cost_array = size_to_cost(battery_size, 1.01*array_size+0.01)
    return cost, cost_battery, cost_array, (cost - cost_battery) / cost, (cost - cost_array) / cost

cost_and_elasticity(200e3, 200e3, 5e6, 10, 5, sol['power'].tolist())

I'm running the gradient descent for more steps with a much smaller amplitude since this code appears to be ~1000 times faster than the equivalent Mathematica. This takes care of some numerical infelicities.

In [ ]:
@njit
def find_minimum_system_cost(
    solar_cost: float,
    battery_cost: float,
    load_cost: float,
    sol: list[float],
) -> tuple[tuple[float, float, float], tuple[float, float, 1], tuple[float, float, float], tuple[float, float, float], any]:
    bi = min(10, 10 * load_cost / 5e6)
    ai = min(10, 10 * load_cost / 5e6)
    amplitude = 10 + 70 * (load_cost / 5e6)
    if 7e5 < load_cost < 13e5: amplitude *= 3
    if 80e6 < load_cost: amplitude *= 0.5

    cost_min = 10**10
    bi_min = bi
    ai_min = ai
    for i in range(100):
        # There was a bug here: cost_and_elasticity were getting called with wrong argument order
        # Changing them around doesn't affect the results since we always passed in the same number here
        cost, cost_bat, cost_arr, dcost_bat, dcost_arr = cost_and_elasticity(solar_cost, battery_cost, load_cost, bi, ai, sol)
        if cost < cost_min:
            ai_min, bi_min, cost_min = ai, bi, cost
        # if True: print((cost, cost_bat, cost_arr, dcost_bat, dcost_arr), bi, ai)
        bi = max(0, bi + amplitude * uniform(0.1, 1) * dcost_bat)
        ai = max(0.01, ai + amplitude * uniform(0.1, 1) * dcost_arr)

    ut = uptime(bi_min / ai_min, 1 / ai_min, sol)
    
    array_cost = solar_cost * ai_min
    storage_cost = battery_cost * bi_min
    total_cost = array_cost + storage_cost + load_cost
    
    return (
        (solar_cost, battery_cost, load_cost),
        (ai_min, bi_min, 1),
        (array_cost, storage_cost, load_cost),
        (array_cost + storage_cost, total_cost, total_cost / ut[-1]),
        ut
    )

find_minimum_system_cost(2e5, 2e5, 1e5, sol['power'].tolist())

That's really not that far from Casey's outcomes. I'm going to chalk the difference up to order of operations and floating point shenanigans.

In [ ]:
from tqdm import tqdm
results_raw = [
    find_minimum_system_cost(200e3, 200e3, 10e3*10**(0.1*i), sol['power'].tolist())
    for i in tqdm(range(40))
]

I'm not sure what's taking Casey's implementation so long? Let's call this a minute, then it's a factor 100? Factor 1000 if I enable compilation with `numba`.


In [ ]:
results_raw

In [ ]:
labels = [
    'solar cost ($/MW)', 'battery cost ($/MW)', 'load cost ($/MW)',
    'array size (MW)', 'battery size (MWh)', 'load size (1 MW by definition)',
    'array cost ($)', 'battery cost ($)', 'load cost ($, normalized to 1 MW)',
    'total power system cost ($)', 'total system cost ($)', 'total cost per utilization ($)',
    'battery size relative to 1 MW array', 'load size relative to 1 MW array', 'annual battery utilization', 'annual load utilization'
]

results = pd.DataFrame([dict(zip(labels, (x for tup in r for x in tup))) for r in results_raw])
results.head()

In [ ]:
subsystems = results.melt(id_vars='annual load utilization', value_vars=[
    'array cost ($)', 'battery cost ($)', 'load cost ($/MW)', 'total power system cost ($)', 'total system cost ($)', 
])
subsystems = subsystems.loc[subsystems['value'] <= 2e7]
alt.Chart(subsystems).mark_line().encode(
    x='annual load utilization:Q',
    y=alt.Y('value:Q', scale=alt.Scale(domain=[0, 2e7], clamp=True)),
    color='variable:N',
    tooltip='variable:N',
).properties(width=800, height=600)

In [ ]:
subsystems = results.melt(id_vars="load cost ($/MW)", value_vars=[
    'array cost ($)', 'battery cost ($)', 'load cost ($/MW)', 'total power system cost ($)', 'total system cost ($)', 
    'total cost per utilization ($)',
])
subsystems = subsystems.loc[subsystems['value'] > 0]

alt.Chart(subsystems).mark_line().encode(
    x=alt.X('load cost ($/MW):Q', scale=alt.Scale(type='log', domain=[0.5e4, 1e8])),
    y=alt.Y('value:Q', scale=alt.Scale(type='log', domain=[0.5e3, 1e8])),
    color='variable:N',
    tooltip=('variable:N','value:Q'),
).properties(width=800, height=600)

In [ ]:
alt.Chart(results).mark_line().encode(
    x=alt.X('load cost ($/MW)', scale=alt.Scale(type='log')),
    y='annual load utilization'
).properties(width=800, height=600)

In [ ]:
results.describe()

In [ ]:
for col in ['array cost ($)', 'battery cost ($)', 'total power system cost ($)']:
    results[col.replace('$', '$/MWh')] = results[col] / (10 * 24 * 365 * results['annual load utilization'])

import numpy as np
min_util = results.loc[results['annual load utilization'].idxmin()]
uvals = np.arange(0.01, min_util['annual load utilization'], 0.001)
underutilized_solar = pd.Series(uvals, name='annual load utilization').to_frame()
underutilized_solar['variable'] = 'underutilized solar'
underutilized_solar['value'] = min_util['array cost ($)'] / (10 * 24 * 365 * uvals)

subsystems = results.melt(id_vars='annual load utilization', value_vars=[
    'array cost ($/MWh)', 'battery cost ($/MWh)', 'total power system cost ($/MWh)', 
])
subsystems = pd.concat([subsystems, underutilized_solar])
alt.Chart(subsystems).mark_line().encode(
    x='annual load utilization:Q',
    y=alt.Y('value:Q', scale=alt.Scale(domain=[0, 80])),
    color='variable:N',
).properties(width=800, height=600).interactive()